In [44]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

#from finanzas import*

In [3]:
#Datos
edad_x=30
sexo="H"
paga= 0.04
salario= 7000000
jubilacion=62
tasa_anual=0.04

In [22]:
pago_mensual= salario*0.16
n=(jubilacion-edad_x)*12
i=tasa_anual/12
total=pago_mensual*n
print(total)
totalint=(pago_mensual*((1+i)**n))
print (totalint)
#vf=vf_cte(pago_mensual,i,n)
#print(vf)

430080000.0
4019671.0900741415


In [39]:
def vf_cte(pago, n, i,valor_inicial):

  pagos = np.ones(n) * pago + np.concatenate(([valor_inicial], np.zeros(n-1)))
  print(pagos.sum())
  cao_factor = np.ones(n) * (1 + i) ** np.arange(n,0,-1)
  resp_vector = pagos * cao_factor
  resp = resp_vector.sum()
  return resp

def TMensual(Tmort):
    """
    Interpola los valores de l(x) en pasos de 1/12 (mensual) y calcula q(x) y p(x).

    Parameters:
    Tmort (DataFrame): DataFrame que contiene las columnas 'x' y 'l(x)'.

    Returns:
    DataFrame: DataFrame con valores interpolados de x, l(x), q(x) y p(x).
    """

    # Aseguramos que Tmort tiene las columnas adecuadas
    if 'x' not in Tmort.columns or 'l(x)' not in Tmort.columns:
        raise ValueError("El DataFrame debe contener las columnas 'x' y 'l(x)'")

    # Obtener los valores de x y l(x)
    x_values = Tmort['x'].values
    lx_values = Tmort['l(x)'].values

    # Crear una función de interpolación
    interp_function = interp1d(x_values, lx_values, kind='linear', fill_value='extrapolate')

    # Crear los nuevos valores de x con un paso de 1/12, hasta 110 (incluyendo 110)
    new_x = np.arange(x_values[0], 110 + 1/12, 1/12)  # Incrementos mensuales hasta 110

    # Calcular los valores de l(x) interpolados
    new_lx = interp_function(new_x)

    # Inicializar los arrays para q(x) y p(x)
    qx_values = np.zeros(len(new_x) - 1)  # Inicializar el array para q(x)
    px_values = np.zeros(len(new_x) - 1)  # Inicializar el array para p(x)

    # Calcular q(x) y p(x) para cada nuevo valor de x
    for i in range(len(new_x)-1):
        if new_lx[i] > 0:  # Evitar división por cero
            qx_values[i] = 1 - (new_lx[i + 1] / new_lx[i])  # q(x) = 1 - l(x + 1/12) / l(x)
        else:
            qx_values[i] = 0  # Si l(x) es 0, q(x) también debe ser 0

        px_values[i] = 1 - qx_values[i]  # p(x) = 1 - q(x)


    # Crear un nuevo DataFrame con los valores interpolados y calculados
    interpolated_df = pd.DataFrame({
        'x': new_x,  # Incluir hasta 110
        'l(x)': new_lx,  # Excluir el último por la misma razón
        'q(x)': np.append(qx_values, 1),  # Añadir el valor de q(110)
        'p(x)': np.append(px_values, 0)  # Añadir el valor de p(110)
    })

    return interpolated_df

def tmort_filtered(df,edad):
    qx_vec=df[df['x']>=edad]
    return qx_vec

In [41]:
Tmort_m= pd.read_excel("/content/TM_mujeres.xlsx")
Tmort_h= pd.read_excel("/content/TM_hombres.xlsx")


In [45]:
T_hombre=TMensual(Tmort_h)
T_mujer=TMensual(Tmort_m)

In [46]:
T_hombre.head(15)

,x,l(x),q(x),p(x)
0,15.000000,1000000.000000,0.000040,0.999960
1,15.083333,999959.583333,0.000040,0.999960
2,15.166667,999919.166667,0.000040,0.999960
3,15.250000,999878.750000,0.000040,0.999960
4,15.333333,999838.333333,0.000040,0.999960
5,15.416667,999797.916667,0.000040,0.999960
6,15.500000,999757.500000,0.000040,0.999960
7,15.583333,999717.083333,0.000040,0.999960
8,15.666667,999676.666667,0.000040,0.999960
9,15.750000,999636.250000,0.000040,0.999960


In [48]:
T=tmort_filtered(T_hombre,jubilacion)
T.head(15)

,x,l(x),q(x),p(x)
564,62.000000,897019.000000,0.000766,0.999234
565,62.083333,896331.500000,0.000767,0.999233
566,62.166667,895644.000000,0.000768,0.999232
567,62.250000,894956.500000,0.000768,0.999232
568,62.333333,894269.000000,0.000769,0.999231
569,62.416667,893581.500000,0.000769,0.999231
570,62.500000,892894.000000,0.000770,0.999230
571,62.583333,892206.500000,0.000771,0.999229
572,62.666667,891519.000000,0.000771,0.999229
573,62.750000,890831.500000,0.000772,0.999228


In [49]:
T["n"] = T["x"].apply(lambda x: x-jubilacion)*12

<ipython-input-49-471fb288521a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T["n"] = T["x"].apply(lambda x: x-jubilacion)*12


In [52]:
T['Vn']=np.ones(len(T))*(1/(1/+i))
T['Vn']=T.apply(lambda x: x['Vn']**x['n'],axis=1)
T

<ipython-input-52-33d20c13056c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['Vn']=np.ones(len(T))*(1/(1/+i))
<ipython-input-52-33d20c13056c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['Vn']=T.apply(lambda x: x['Vn']**x['n'],axis=1)


,x,l(x),q(x),p(x),n,Vn
564,62.000000,897019.000000,0.000766,0.999234,4.007461e-12,1.000000e+00
565,62.083333,896331.500000,0.000767,0.999233,1.000000e+00,3.333333e-03
566,62.166667,895644.000000,0.000768,0.999232,2.000000e+00,1.111111e-05
567,62.250000,894956.500000,0.000768,0.999232,3.000000e+00,3.703704e-08
568,62.333333,894269.000000,0.000769,0.999231,4.000000e+00,1.234568e-10
...,...,...,...,...,...,...
1136,109.666667,121.333333,0.083104,0.916896,5.720000e+02,0.000000e+00
1137,109.750000,111.250000,0.090637,0.909363,5.730000e+02,0.000000e+00
1138,109.833333,101.166667,0.099671,0.900329,5.740000e+02,0.000000e+00
1139,109.916667,91.083333,0.110704,0.889296,5.750000e+02,0.000000e+00


In [54]:
sum_vect=T['p(x)']*T['Vn']
suma=sum_vect.sum()
suma=float(suma)
suma

1.002575488840214

### EJERCICIO

In [55]:
jubilacion=57
pago_mensual=salario*0.16
n=(jubilacion - edad_x)*12
i=tasa_anual/12
valor_i=0
vf=vf_cte(pago_mensual,n,i,valor_i)
print(vf)
print(n*pago_mensual)
T=tmort_filtered(T_mujer,jubilacion)
T['n']=T['x'].apply(lambda x: x-jubilacion)*12
T['Vn']=np.ones(len(T))*(1/(1/+i))
T['Vn']=T.apply(lambda x: x['Vn']**x['n'],axis=1)
sum_vect=T['p(x)']*T['Vn']
suma=sum_vect.sum()
print(suma)

362880000.0
653809052.6404626
362880000.0
1.0030666894661313


<ipython-input-55-b85bb0774ede>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['n']=T['x'].apply(lambda x: x-jubilacion)*12
<ipython-input-55-b85bb0774ede>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['Vn']=np.ones(len(T))*(1/(1/+i))
<ipython-input-55-b85bb0774ede>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [56]:
print(vf/suma)

651810153.3093913


In [57]:
jubilacion=57
pago_mensual=salario*0.16
n=(jubilacion - edad_x)*12
i=tasa_anual/12
valor_i=96000000
vf=vf_cte(pago_mensual,n,i,valor_i)
print(vf)
print(n*pago_mensual)
T=tmort_filtered(T_mujer,jubilacion)
T['n']=T['x'].apply(lambda x: x-jubilacion)*12
T['Vn']=np.ones(len(T))*(1/(1+i))
T['Vn']=T.apply(lambda x: x['Vn']**x['n'],axis=1)
sum_vect=T['p(x)']*T['Vn']
suma=sum_vect.sum()
print(suma)

458880000.0
935991032.5096068
362880000.0
263.2164186769155


<ipython-input-57-27676e34d352>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['n']=T['x'].apply(lambda x: x-jubilacion)*12
<ipython-input-57-27676e34d352>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['Vn']=np.ones(len(T))*(1/(1+i))
<ipython-input-57-27676e34d352>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [58]:
print(vf/suma)

3555975.1067751106
